In [ ]:
%%capture

!pip install -q pyspark findspark

import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
!git clone https://github.com/javieraespinosa/big-data-analytics-practicals
!mv big-data-analytics-practicals/data/* .
!ls *.csv

In [ ]:
sc = spark.sparkContext
sc

# TP RDD

Dans cet atelier, nous allons explorer certains les concepts de RDD que nous avons abordés. Nous utiliserons un ensemble de données comprenant les crimes signalés à Washington, du 3 octobre 2015 au 2 octobre 2016. Ces données proviennent du catalogue de données ouvertes du district de Columbia. Nous utiliserons ces données pour explorer certaines transformations et actions sur les RDD et les pairs RDD.
https://github.com/frankieliu/databricks2/blob/master/wash_dc_crime_incidents_2013.csv

Nous pouvons à présent commencer le TP.

## Créer un RDD

La première étape consiste à charger les données.
Exécutez la cellule suivante pour créer un RDD contenant les données.

In [ ]:
myRdd =  sc.textFile("wash_dc_crime_incidents_2013.csv")

La lecture d'un fichier n'est pas une action et par conséquent aucun traitement sera effectué.

Comptez le nombre de ligne contenu dans le RDD `myRDD` :

## Exploration des données

Voyons quelques-unes des données.

In [ ]:
# Affichez les 5 premières lignes du RDD "myRdd"


Super, comme vous pouvez le constater il y a un en-tête.
Cet en-tête est fourni au début du fichier, cela signifie qu'une seul partition disposera de l'en-tête.
Pour effectuer un traitement homogène sur l'ensemble des données, nous devrons vérifier si le bloc en cours de traitement a un en-tête.
Quand le fichier taille des Go voir de To de données, cette vérification peut vite devenir coûteuse, c'est pourquoi on préfère traiter les données sans en-tête.

Trouvez une solution pour supprimer l'en-tête :

Indice: utilisez la method filter de l'API RDD.

https://spark.apache.org/docs/latest/api/python/pyspark.html?

In [ ]:
# indiquer le traitement à effectuer pour supprimer l'en-tête
# nous appelerons le RDD sans en-tête "noHeaderRdd"


Sauvegarder les données sans l'entête sous le nom de fichier wash_dc_crime_incidents_2013_without_header.csv

In [ ]:
noHeaderRdd.saveAsTextFile('wash_dc_crime_incidents_2013_without_header.csv')

A partir du fichier sans entête, découper les lignes en cellules individuelles.

Nous appelerons le RDD resultant de cette transformation "CrimeData".


In [ ]:
# Complétez le code


La réprésentation des données laisse à désirer sans la schéma des données.
En python, on peut associer des noms à chaque valeur du tuple avec le module namedtuple :

In [ ]:
from collections import namedtuple

CrimeDataTuple = namedtuple('CrimeData', ['date_string', 'time_string', 'offense', 'latitude', 'longitude'])

def map_line(line):
  cols = line.split(",")
  return CrimeDataTuple(date_string=cols[10], time_string=cols[11], offense=cols[3], latitude=cols[7], longitude=cols[8])

CrimeDataT = noHeaderRdd.map(map_line)

CrimeDataT.take(10)

On obtient une liste de tuple plus lisible.

A présent, regroupez les données par type de crime (OFFENSE)

In [ ]:
# Nous utiliserons la methode groupBY


In [ ]:
#Affichez les 5 premiers éléments :


Ensuite, créez un RDD qui compte le nombre de chaque infraction (OFFENSE).

In [ ]:
#Indice : utilisez la méthode map sur le RDD pair obtenu précédement avec la function len


A partir du RDD CrimeDataT, répéter l'opération de comptage des OFFENSES avec la méthode RDD groupByKey :

In [ ]:
#Indice : utilisez la méthode map sur le RDD pair obtenu précédement avec la function len


Combien de meurtres (homicide) y a-t-il eu pendant la période couverte par les données?

Attention votre traitement devra être insensible à la casse

In [ ]:
# utilisez le RDD CrimeDataT pour compter le nombre d'homicide
# votre traitement devra être insensible à la maniére dont est ecrit homicide
# Vous devrez utiliser effectuer le comptage avec la méthode reduceByKey
# Vous afficherez le résultat de votre RDD homicide_count


Affichez le top 5 des crimes :

Vous pouvez à présent fermer le SparkContext.

In [ ]:
sc.stop()